# Use cases: Connection requests

In [1]:
import pandas as pd
import numpy as np
import copy
import pandapower as pp
import os,sys,inspect
import json
import math
import yaml
import folium
from folium.plugins import BeautifyIcon
import plotly.graph_objs as go

#import packages from parent subfolder 
currentdir = os.path.abspath(os.getcwd())
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 


In [2]:
def distance_to_poi(poi, df):
    """
    Calculate the Haversine distance.

    Parameters
    ----------
    origin : tuple of float
        (lat, long)
    destination : 

    Returns
    -------
    distance_in_km : float

    """
    lat1, lon1 = poi
    closest_point = None
    min_d = None
    df['distance to poi'] = None

    for idx,row in df.iterrows():
        lat2 = row['y']
        lon2 = row['x']
        
        radius = 6371  # km

        dlat = math.radians(lat2 - lat1)
        dlon = math.radians(lon2 - lon1)
        a = (math.sin(dlat / 2) * math.sin(dlat / 2) +
                math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) *
                math.sin(dlon / 2) * math.sin(dlon / 2))
        c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
        d= radius * c
        df['distance to poi'].loc[idx] = d
  

    return df

## Data Preparation


In [3]:
# Pandapower grid
net = pp.from_json(os.path.join(currentdir,'data\svedala\svedala.json')) 
#Read headroom 
head_load= pd.read_json(os.path.join(currentdir,'data\svedala\svedala_headroom.json'))
head_sgen= pd.read_json(os.path.join(currentdir,'data\svedala\svedala_headroom_sgen.json'))

In [4]:
# Data preparation
df = copy.deepcopy(net.bus_geodata)

#Need to adjust coordinates (NOT NEEDED WITH REAL DATA)
df['x'] = df['x']*0.1 + 13.5
df['y'] = df['y']*0.1 + 57.4

df['sgen'] = head_sgen['Headroom']
df['load'] = head_load['Headroom']
df['voltage'] = net.bus.vn_kv
df = df.drop('coords', 1)

## Use case 1
    "As a new customer, 
    I want to see whether I can connect at a grid connection point"

For this use case the customer has a connection request in mind. The customer would like to connect a load or generator of a specific size and would want to see on a map where it is possible to connect.

Vision is that this information should be public available meaning only necessary information should be published. In this case the customer is only intrested in available headroom (here its dependent on type of connection) and position. For understanding which type of connection is available we have choosen to also include voltage limits, but this could of course be omited if needed.


In [5]:
# Customer requested connection size and type
size = 50 # in MW
connection_type = 'sgen' #'load' or 'sgen'

In [6]:
#Data for use case:
df.head()

,x,y,sgen,load,voltage
11,13.649830,57.628391,140.62500,34.375,135.0
12,13.797293,57.560715,105.46875,56.250,17.0
13,13.733433,57.533442,500.00000,56.250,400.0
14,13.837288,57.524723,250.00000,56.250,17.0
21,13.176039,56.861547,3.90625,0.000,11.0


The example data, shown above, needs to be decorated based on the customer request. For example position on the map should change color depending on if the request is possible or not, the marker will change shape and color based on voltage level.

In [7]:
# Create map layer depending on customer request
icon = []
for idx,row in df.iterrows():
    if row[connection_type] < size:
        color= 'red'
    else:
        color= 'green'
    
    #shape
    if row['voltage']<50:
        shape = 'circle-dot'
        width = 4
    else:
        shape = 'rectangle-dot'
        width = 6
    
    icon.append(BeautifyIcon(border_color=color,border_width=width,icon_shape=shape, prefix='fa'))

df['icon'] = icon
map2 = folium.Map(
    location=[57.772080,12.779226], 
    tiles='cartodbpositron',
    zoom_start=9,
)


df.apply(lambda row:folium.Marker(location=[row["y"], row["x"]],tooltip=row[connection_type],icon=row['icon']).add_to(map2), axis=1)
map2

## Use case 2
    "As a connection request handler,
    I want to be able to quickly give preliminary answers to new connection requests.
"

For this use case the customer request handler recives a request from a customer. The request will include connection type, connection size and geographical coordinates. From the coordinates the user want to find the nearest connection point and see if the connection size can be added to this point.

In [8]:
size = 45
poi = (57.172080,13.449226)#svedala.grid.bus.index of intrested connections
connection_type = 'load'
connection_request = pd.DataFrame(columns = ['x','y','request'], data = [[poi[0],poi[1],size]])

In [9]:
#Calculate distance from point of intrest (poi) to all possible connections points and sort on distance
df = distance_to_poi(poi, df)
df_sorted = df.sort_values(by=['distance to poi'])

In [10]:
# Create map layer depending on customer request
map = folium.Map(
    location=[poi[0],poi[1]], 
    tiles='cartodbpositron',
    zoom_start=9,
)
icon = []
for idx,row in df.iterrows():

    #highlight closest 3 connections
    if idx in df_sorted.head(3).index[:]:
        color = 'red' if row[connection_type] < size else 'green'
        #shape = 'circle-dot' if row['voltage']<50 else 'rectangle-dot'
        icon.append(BeautifyIcon(border_color=color,border_width=2,icon_shape='doughnut', prefix='fa'))
        popup = 'Distance '+ str(df_sorted.loc[idx,'distance to poi'])
        folium.PolyLine([[row["y"], row["x"]],[connection_request.loc[0,'x'],connection_request.loc[0,'y']]], color="blue", weight=4, opacity=1,tooltip=popup).add_to(map)

        continue
    if row[connection_type] < size:
        color= 'red'
    else:
        color= 'green'
    
    #shape
    if row['voltage']<50:
        shape = 'circle-dot'
        width = 4
    else:
        shape = 'rectangle-dot'
        width = 4
    
    


    icon.append(BeautifyIcon(border_color=color,border_width=width,icon_shape=shape, prefix='fa'))

df['icon'] = icon



df.apply(lambda row:folium.Marker(location=[row["y"], row["x"]],tooltip=row[connection_type],icon=row['icon']).add_to(map), axis=1)


connection_request.apply(lambda row:folium.Marker(location=[row["x"], row["y"]],tooltip='Request size '+str(row['request']),
                                       color='blue',fill=True).add_to(map), axis=1)
map